In [8]:
import pandas as pd
from collections import Counter
import numpy as np

date = pd.read_excel('tema_2_date_interviu.xlsx')
date = date.fillna("empty")
date.columns

Index(['COMPANIA', 'INDUSTRIA', 'ID_CANDIDAT', 'SEX', 'LOCATIA_COMPANIEI',
       'LOCATIA_CANDIDATULUI', 'RASPUNS_LA_TELEFON', 'A_VENIT_LA_INTERVIU',
       'STARE_CIVILA'],
      dtype='object')

In [9]:
class CLASIFICATOOOORUL():

    def __init__(self, date, no_labels):
        
        # aici stocam tabelul initial cu toate informatiile pe care le avem
        self.data = date 
        self.no_labels = no_labels
        self.prior = [1/no_labels for _ in range(no_labels)]
        self.yes_counter = 0
        self.no_counter = 0
        
        # selectez toate informatiile despre candidati (ignor variabila pe care trebuie sa o prezic si id-ul pentru ca nu este relevant)
        # Poate ar trebui sa ignor si alte variabile
        self.coloane_de_interes = [column for column in self.data.columns if column not in ['A_VENIT_LA_INTERVIU', 'ID_CANDIDAT']]
        
        
        """
        Pentru fiecare coloana (COMPANIE, INDUSTRIE, SEX, etc.) construiesc un dictionar uite voi stoca toate probabilitatile. De exemplu pentru
        coloanele din paranteze dictionarul meu va arata asa:
        self.prob_map = {
            "COMPANIE": {}, # dictionar gol unde voi pune probabilitati
            "INDUSTRIE": {}, # alt dictionar gol unde voi pune probabilitati (veti pune)
            "SEX": {}, # alt dictionar gol. si asa mai departe.
        
        }
        Pentru ca trebuie sa prezicem doar doua valori (YES/NO) avem nevoie de cate doua dictionare pentru fiecare coloana.
        Puteti sa printati ambele dictionare ca sa vedeti ce continut au. (Vedeti urmatoarea celula)
        """
        self.yes_prob_map = {column: {} for column in self.coloane_de_interes}
        self.no_prob_map = {column: {} for column in self.coloane_de_interes}
        
        """
        Avem de asemenea nevoie de niste liste in care sa tinem minte ce cuvinte am intalnit in cazurile in care au venit
        la interviu, cat si in cazurile in care nu au venit. Am pus aceste liste intr-un dictionar ca sa le pot identifica usor
        """
        self.yes_words = {column: [] for column in self.coloane_de_interes}
        self.no_words = {column: [] for column in self.coloane_de_interes}
    
    
    def fit(self):
        
        self.adauga_cuvinte_in_lista()
        self.calculeaza_probabilitati(self.yes_words, self.yes_prob_map, self.yes_counter)
        self.calculeaza_probabilitati(self.no_words, self.no_prob_map, self.no_counter)
                
    def adauga_cuvinte_in_lista(self):
        
        for key, rand in self.data.iterrows():
            if rand['A_VENIT_LA_INTERVIU'] == 'Yes':
                # de completat: iterez_peste_rand si actualizez yes_words
                self.itereaza_peste_rand(rand, self.yes_words)
                # trebuie sa actualizam si counterul
                self.yes_counter += 1
            else:
                # de completat: iterez_peste_rand si actualizez no_words
                self.itereaza_peste_rand(rand, self.no_words)
                self.no_counter += 1 # trebuie sa actualizam si counterul
    
    @staticmethod
    def calculeaza_probabilitati(liste, prob_map, total_counter):
        
        """
        Liste este un dictionar care are coloane drep chei si liste de cuvinte drept valori. Vezi exemplele de poze de mai jos. 
        Trebuie sa trecem prin fiecare lista, sa numaram cate aparitii sunt si sa actualizam dictionarul de probabiliati.
        """
        for coloana in liste: 
            counter_dict = Counter(liste[coloana]) # 
            for cuvant, numar_aparitii in counter_dict.items():
                # de completat: trebuie sa calculam probabilitatea pentru fiecare cuvant din coloana si sa ne raportam la cate YES/NO avem.
                prob_map[coloana][cuvant] = numar_aparitii/total_counter
                
    def itereaza_peste_rand(self, rand, lista_cuvinte):
        """
        Iteram peste fiecare coloana de interes si inseram cuvintele in liste. Randurile vor avea ca index numele coloanelor,
        deci putem sa selectam o valoare folosind 'nume_coloana'.
        """
        for nume_coloana in self.coloane_de_interes:
            # de completat
            lista_cuvinte[nume_coloana].append(rand[nume_coloana])
            
        
    def predict(self, rand_din_data_frame):
        
        logs = self.inference(rand_din_data_frame)
        
        return np.argmax(logs)
    
    def inference(self, rand_din_data_frame):
        """
        rand_din_data_frame: primim pentru predictie un rand si returnam valorile pentru yes and no.
        """
        yes_sum = 0
        no_sum = 0
        for coloana in self.coloane_de_interes:
            valoare_din_coloana = rand_din_data_frame[coloana]
            
            if valoare_din_coloana not in self.yes_prob_map[coloana]:
                yes_sum += np.log(0.00001) # daca nu am cuvant acolo, adaug o probabilitate de forma.
            else:
                yes_sum += np.log(self.yes_prob_map[coloana][valoare_din_coloana])
                
            if valoare_din_coloana not in self.no_prob_map[coloana]:
                no_sum += np.log(0.00001) # daca nu am cuvant acolo, adaug o probabilitate de forma.
            else:
                no_sum += np.log(self.no_prob_map[coloana][valoare_din_coloana])
        
        yes_sum += np.log(self.prior[1])
        no_sum += np.log(self.prior[0])
        
        # nu trebuie sa transform inapoi folosind exp pentru ca logaritmul pastreaza ordinea. Adica de exemplu,
        # daca no_sum este mai mare, va fi mai mare chiar daca aplic log.
        return [no_sum, yes_sum]
    

In [12]:
# clasificatoooorul = CLASIFICATOOOORUL(date, 2)
print('------ Dictionarul de probabilitati ------')
print()
print(clasificatoooorul.yes_prob_map) # unul pentru YES (VINE LA INTERVIU)
print(clasificatoooorul.no_prob_map) # unul pentru NO (NU VINE LA INTERVIU)


print('------ Listele de cuvinte ------' )
print()
print(clasificatoooorul.yes_words)
print(clasificatoooorul.no_words)

------ Dictionarul de probabilitati ------

{'COMPANIA': {'Aon Hewitt': 0.03422619047619048, 'UST': 0.01488095238095238, 'Standard Chartered Bank': 0.7291666666666666, 'ANZ': 0.02976190476190476, 'Hospira': 0.041666666666666664, 'Pfizer': 0.0625, 'Standard Chartered Bank Chennai': 0.010416666666666666, 'Aon hewitt Gurgaon': 0.002976190476190476, 'Astrazeneca': 0.008928571428571428, 'Flextronics': 0.017857142857142856, 'Prodapt': 0.008928571428571428, 'Williams Lea': 0.011904761904761904, 'Barclays': 0.00744047619047619, 'Hewitt': 0.017857142857142856, 'Woori Bank': 0.001488095238095238}, 'INDUSTRIA': {'IT Services': 0.020833333333333332, 'BFSI': 0.7782738095238095, 'Pharmaceuticals': 0.1130952380952381, 'IT Products and Services': 0.049107142857142856, 'Electronics': 0.017857142857142856, 'Telecom': 0.008928571428571428, 'IT': 0.011904761904761904}, 'SEX': {'Male': 0.7842261904761905, 'Female': 0.21577380952380953}, 'LOCATIA_COMPANIEI': {'Gurgaon': 0.03571428571428571, 'Bangalore': 0.2

#### Aveti aici doua imagini unde am aratat rezultatul calculelor

Am atasat doua print screens. Cam asa ar trebui sa arate dicionarele de mai sus dupa ce calculam probabilitatile.


![Liste de Cuvinte](rezultat_liste_cuvinte.png)

![Liste de probabilitati](rezultat_liste_probabilitati.png)

In [11]:
clasificatoooorul = CLASIFICATOOOORUL(date, 2)
clasificatoooorul.fit()

In [13]:

def evaluate_model(model, data):
    
    from sklearn.metrics import  classification_report
    from tqdm import tqdm
    
    predictions = []
    true_values = []
    for key, row in tqdm(data.iterrows()):
        predictions.append(model.predict(row)) # aici adaug in lista predictia modelului
        true_values.append(1 if row['A_VENIT_LA_INTERVIU'] == 'Yes' else 0) # aici adaug in lista 1 daca este YES sau 0 daca este NO
        
    print(classification_report(true_values, predictions))
    
evaluate_model(clasificatoooorul, date)

965it [00:00, 13532.50it/s]

              precision    recall  f1-score   support

           0       0.38      0.70      0.49       293
           1       0.79      0.50      0.61       672

    accuracy                           0.56       965
   macro avg       0.58      0.60      0.55       965
weighted avg       0.67      0.56      0.57       965




##### Scorul nu da foarte bine. Incercati sa mai eliminati din coloane. Poate avem prea multa informatie neimportanta. Incercati sa gasiti un set de coloane care da un scor mai bun.
